In [1]:
import time
import tracemalloc

In [2]:
def cross(A,B):
    return [a+b for a in A for b in B]

In [3]:
def sudoku(puzzle):
    """ 
        Sudoku must be entered as a string 81 characters long.
        Unfilled cells must contain a full-stop 
    """
    
    rows = 'ABCDEFGHI'
    columns = '123456789'
    digits = '123456789'
    
    variables = cross(rows, columns)
    domains = dict(zip(variables, puzzle))
    
    for key, values in domains.items():
        if values == '.':
            domains[key] = list(digits)
        else:
            domains[key] = list(domains[key])
     
    unitlist = [cross(rows,a) for a in columns] + [cross(a, columns) for a in rows] + [cross(a,b) for a in ['ABC', 'DEF', 'GHI'] for b in ['123', '456', '789']]
    
    units = dict((s,[t for t in unitlist if s in t]) for s in variables)
    
    neighbours =  dict((s, set(units[s][0] +units[s][1]+ units[s][2]) - set([s])) for s in variables)
    
    constraints = lambda X, x, Y, y : x != y
    
    return variables, domains, neighbours, constraints

In [4]:
class CSP():
    def __init__(self, variables, domains, neighbours, constraints):
        self.variables = variables #list of variables
        self.domains = domains #dictionary where keys are variables
        self.neighbours = neighbours #dict: keys are variables, values are lists other variables in constraints with key
        self.constraints = constraints #function X,x,Y,y returns true if X = x, Y = y satisfy constraints
                                        #e.g. lambda X,x,Y,y : x != y
        self.nNodes = 0 #count number of nodes in search tree
                        
    def prune(self, var, value):
        """ romove value val from domain of variable var """
        self.domains[var].remove(value)
        
    def assign(self, var, val, assignment):
        """ add variable var and value val to assigment """
        assignment[var] = val
        self.nNodes += 1
        
    def unassign(self, var, assignment):
        """ remove variable var from assignment """
        if var in assignment:
            del assignment[var]
           
    def nConflicts(self, var, val, assignment):
        """ return number of conflicts var = val has in assignent """
        count = 0
        for var2 in self.neighbours[var]:
            if var2 in assignment and not self.constraints(var, val, var2, assignment[var2]):
                count += 1
        return count
    
    def isGoalState(self, assignment):
        """ check is the current state a goal state """
    
        result = (len(assignment) == len(self.variables)) and (all(self.nConflicts(var, assignment[var], assignment) == 0 for var in self.variables))
        return result
    
    
    def display(self, assignment):
        def show_box(box): return [' '.join(map(show_cell, row)) for row in box]

        def show_cell(cell): return str(assignment.get(cell, '.'))

        def abut(lines1, lines2): return list(
            map(' | '.join, list(zip(lines1, lines2))))

        print('\n------+-------+------\n'.join(
            '\n'.join(reduce(
                abut, map(show_box, brow))) for brow in self.bgrid))

In [5]:
def revise(csp, Xi, Xj):
    revised = False
    for x in csp.domains[Xi]:
        if all(not csp.constraints(Xi,x,Xj,y) for y in csp.domains[Xj]):
            csp.prune(Xi, x) 
            revised = True
    return revised

In [6]:
def AC3(csp):
    queue = {(Xi, Xj) for Xi in csp.variables for Xj in csp.neighbours[Xi]}
    while queue:
        (Xi, Xj) = queue.pop()
        if revise(csp, Xi, Xj):
            if not csp.domains[Xi]:
                return False
            for Xk in csp.neighbours[Xi]:
                if Xk != Xj:
                    queue.add((Xk, Xi))
    return True

In [7]:
def count(seq):
    """Count the number of items in sequence that are interpreted as true."""
    return sum(map(bool, seq))

In [8]:
def num_legal_values(csp, var, assignment):
        return count(csp.nConflicts(var, val, assignment) == 0 for val in csp.domains[var])

In [9]:
def mrv(assignment, csp):
    """Minimum-remaining-values heuristic."""
    return min([v for v in csp.variables if v not in assignment],
                             key=lambda var: num_legal_values(csp, var, assignment))

In [10]:
def lcv(var, assignment, csp):
    """Least-constraining-values heuristic."""
    return sorted(csp.domains[var], key=lambda val: csp.nConflicts(var, val, assignment))

## Edit this function for adding and removing different heuristics

In [11]:
def backtracking_search(csp):
    
  
    #AC3(csp)
    assignment = {}
    for var in csp.variables:
        if (len(csp.domains[var]) == 1):
            assignment[var] = csp.domains[var]
          
    def backtrack(assignment):
        if len(assignment) == len(csp.variables):
            return assignment #if len(assignment) == 81, return assignment. Do goal test. If goal return True. Else assertion error
        
        #now if len(assignment) < 81, i.e. partial assignment enter for loop.
        
        #var = next(iter([v for v in csp.variables if v not in assignment]), None) #var = next variable from variables list that isn't in assignment
        
        #add in MRV heuristic
        #might take less steps but each step takes longer 
        var = mrv(assignment, csp)#min([v for v in csp.variables if v not in assignment], key = lambda v : len(csp.domains[v]), default = None)
        
        for value in csp.domains[var]: #lcv(var, assignment, csp):# #cycle through remaining domain vales for var
            if 0 == csp.nConflicts(var, value, assignment):
                csp.assign(var, value, assignment)                
                result = backtrack(assignment) #runs new backtrack with new var and val. don't get confused between these 
                if result is not None:
                    return result
        csp.unassign(var, assignment) #unassigns previously assigned variable
        return None

    result = backtrack(assignment)
    assert result is None or csp.isGoalState(result) 

    return result

## enter name of data set here

In [12]:
file = open("50easiest.txt", "r")
puzzles = file.readlines()

## run this cell to run algorithm and get stats

In [13]:
steps = []
solvable = []
times = []
memories = []
for puzzle in puzzles:
    variables, domains, neighbours, constraints = sudoku(puzzle[0:81])
    problem = CSP(variables, domains, neighbours, constraints)
    
    mem = tracemalloc
    mem.start()
    start = time.time()
    
    solution = backtracking_search(problem)
    
    duration = time.time() - start
    current, peak = tracemalloc.get_traced_memory()
    mem.stop()
    
    solved =  problem.isGoalState(solution)
    solvable.append(solved)
    steps.append(problem.nNodes)  
    times.append(duration)
    memories.append(peak)
    
print("Average number of nodes : {:.2f}".format(sum(steps)/50))
print("Maximum number of nodes : {:.2f}".format(max(steps)))
print("Average time (seconds) : {:.4f}".format(sum(times) / 50))
print("Maximum time (seconds) : {:.4f}".format(max(times)))
print("Average memory : {:.4f}".format(sum(memories)/50/(10**6)))
print("Maximum memory : {:.4f}".format(max(memories)/(10**6)))

Average number of nodes : 52.64
Maximum number of nodes : 59.00
Average time (seconds) : 0.0362
Maximum time (seconds) : 0.0469
Average memory : 0.0092
Maximum memory : 0.1556
